In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import datetime

from IPython.display import display
pd.options.display.max_columns = None

In [16]:
# with open('F:/DataScience/test_v2.csv','r',encoding='utf-8') as f:
#     with open('F:/DataScience/test_v2_1.csv','w',encoding='utf-8') as f2:
#         for line in f:
#             f2.write(line.replace('\'','\"'))
       

In [40]:
# f= open('F:/DataScience/flatten_train_v2.csv','r',encoding='utf-8')
# r = f.readlines()
    

In [65]:
pd.concat([pd.DataFrame([r[100000].split(',')], columns=r[0].split(',')),pd.DataFrame([r[100002].split(',')], columns=r[0].split(',')+['miss'])])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,channelGrouping,date,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,fullVisitorId,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,miss,socialEngagementType,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime
0,Organic Search,20170529,Chrome,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,not available in demo dataset,not available in demo dataset,3623819892424331961,not available in demo dataset,not available in demo dataset,Europe,Czechia,not available in demo dataset,not available in demo dataset,not available in demo dataset,mjanik.net,not available in demo dataset,not available in demo dataset,Eastern Europe,NaN,Not Socially Engaged,,15,1,13,,284,,,,1,,,not available in demo dataset,,,,,(not set),,(not provided),organic,,google\n,1496073409,1,1496073409
0,Organic Search,20170529,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,5863936343673668667,Bengaluru,not available in demo dataset,Asia,India,not available in demo dataset,not available in demo dataset,(not set),122-airtelbroadband.in,not available in demo dataset,Karnataka,Southern Asia,google\n,Not Socially Engaged,,15,,12,,2289,,,,1,,,not available in demo dataset,,,,,(not set),,True,(not provided),organic,,1496087011,2,1496087011


In [51]:
r[100003]

'Organic Search,20170529,9993197829693068679,Not Socially Engaged,1496099391,9,1496099391,Safari,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,iOS,not available in demo dataset,not available in demo dataset,not available in demo dataset,San Jose,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,San Francisco-Oakland-San Jose CA,comcast.net,not available in demo dataset,California,Northern America,,15,,10,,340,,,,1,,,not available in demo dataset,,,,,(not set),,True,(not provided),organic,,google\n'

In [2]:
def partially_load_df(csv_path='train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    for i,df in enumerate(pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},   # Important!!
                     nrows= nrows, chunksize=100000)):
        
        for column in JSON_COLUMNS:
       
            column_as_df = json_normalize(df[column])
            column_as_df.index = df.index
            column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index = True, left_index = True)
        #dropping hits column
        df.drop(['hits'], inplace=True, axis=1)
        if 'trafficSource.campaignCode' in df.columns:
            df.drop(['trafficSource.campaignCode'], inplace=True, axis=1)
        if i==0:
            df.to_csv(f"{os.path.dirname(csv_path)}/2_flatten_{os.path.basename(csv_path)}", index=False, mode = "a")
        else:
            df.to_csv(f"{os.path.dirname(csv_path)}/2_flatten_{os.path.basename(csv_path)}", index=False, mode = "a", header=False)
    
        print(f"Loaded Shape: {df.shape}")


In [4]:
partially_load_df("F:/DataScience/test_v2.csv")


Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (1589, 58)


In [3]:
%%time
partially_load_df("F:/DataScience/train_v2.csv")

Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (100000, 58)
Loaded Shape: (8337, 58)
Wall time: 11min 54s


In [89]:
partially_load_df("F:/DataScience/train_v2.csv",200000)


device
geoNetwork
totals
trafficSource
Index(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId', 'hits',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.browser', 'device.browserSize', 'device.browserVersion',
       'device.deviceCategory', 'device.flashVersion', 'device.isMobile',
       'device.language', 'device.mobileDeviceBranding',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.screenColors', 'device.screenResolution', 'geoNetwork.city',
       'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits',

In [75]:
df_train = pd.read_csv('F:/DataScience/1flatten_train_v2.csv', sep=',',encoding ='utf8',dtype={'fullVisitorId': 'str'},skiprows=1000 , nrows=20)

In [80]:
df_train.head()

,Referral,20171016,3231434154450089657,Not Socially Engaged,1508167270,1,1508167270.1,Chrome,not available in demo dataset,not available in demo dataset.1,mobile,not available in demo dataset.2,True,not available in demo dataset.3,not available in demo dataset.4,not available in demo dataset.5,not available in demo dataset.6,not available in demo dataset.7,not available in demo dataset.8,Android,not available in demo dataset.9,not available in demo dataset.10,not available in demo dataset.11,not available in demo dataset.12,not available in demo dataset.13,Americas,Mexico,not available in demo dataset.14,not available in demo dataset.15,not available in demo dataset.16,inext.net.mx,not available in demo dataset.17,not available in demo dataset.18,Central America,1.1,1.2,1.3,1.4,1.5,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,1.6,Unnamed: 44,Unnamed: 45,not available in demo dataset.19,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,(not set),Unnamed: 52,Unnamed: 53,referral,/permissions/using-the-logo.html,google.com
0,Affiliates,20171016,8096701741851830466,Not Socially Engaged,1508203382,1,1508203382,Opera Mini,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Africa,Ethiopia,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,Eastern Africa,1,1,1.0,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,Data Share Promo,NaN,NaN,affiliate,NaN,Partners
1,Display,20171016,581895914272108819,Not Socially Engaged,1508217495,1,1508217495,Safari (in-app),not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,iOS,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,Northern America,1,1,1.0,1,1,NaN,NaN,NaN,NaN,1,Google Merchandise Store,Content,not available in demo dataset,CKGMuP3z9tYCFQ2ffgodtL8GPQ,False,1.0,RHS,1000557 | GA | US | en | Hybrid | GDN Text+Ban...,NaN,(User vertical targeting),cpc,NaN,google
2,Affiliates,20171016,1460355630680560111,Not Socially Engaged,1508151687,1,1508151687,Samsung Internet,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,not available in demo dataset,not available in demo dataset,Paris,not available in demo dataset,Europe,France,not available in demo dataset,not available in demo dataset,(not set),fr.eu,not available in demo dataset,Ile-de-France,Western Europe,1,1,1.0,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,Data Share Promo,NaN,NaN,affiliate,NaN,Partners
3,Affiliates,20171016,8089976046233396489,Not Socially Engaged,1508171754,1,1508171754,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available

In [77]:
df_train2 = pd.read_csv('F:/DataScience/1flatten_train_v2.csv', sep=',',encoding ='utf8',dtype={'fullVisitorId': 'str'}, nrows=20)

In [83]:
df_train2.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'socialEngagementType',
       'visitId', 'visitNumber', 'visitStartTime', 'device.browser',
       'device.browserSize', 'device.browserVersion', 'device.deviceCategory',
       'device.flashVersion', 'device.isMobile', 'device.language',
       'device.mobileDeviceBranding', 'device.mobileDeviceInfo',
       'device.mobileDeviceMarketingName', 'device.mobileDeviceModel',
       'device.mobileInputSelector', 'device.operatingSystem',
       'device.operatingSystemVersion', 'device.screenColors',
       'device.screenResolution', 'geoNetwork.city', 'geoNetwork.cityId',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.metro', 'geoNetwork.networkDomain',
       'geoNetwork.networkLocation', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.sessionQualityDim',
       'totals.tim